In [1]:
%pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df = pd.read_csv(r'C:\Users\rajwi\OneDrive\Desktop\harvard’s artifacts project\chocolate_products_cleaned1.csv') 
display(df)

,brand,product_code,product_name,energy-kcal_value,energy-kj_value,carbohydrates_value,sugars_value,fat_value,saturated-fat_value,proteins_value,fiber_value,salt_value,sodium_value,nova-group,nutrition-score-fr,fruits-vegetables-nuts-estimate-from-ingredients_100g,sugar_to_carb_ratio,calorie_category,sugar_category,is_ultra_processed
0,Sidi Ali,6111035000430,Sidi Ali,0.0,20.0,42.0,14.00,0.0,0.00,0.0,0.0,0.00000,0.000000,4.0,1.0,0.00000,0.333333,Low,Low Sugar,Yes
1,Jaouda,6111242100992,Perly,97.0,406.0,9.4,4.10,3.0,1.40,8.0,2.5,0.38000,0.152000,3.0,7.0,0.00000,0.436170,Low,Low Sugar,No
2,sidi ali,6111035002175,Sidi Ali,241.0,1190.0,15.0,4.10,6.1,1.40,5.7,2.5,65.00000,26.000000,4.0,0.0,0.00000,0.273333,Moderate,Low Sugar,Yes
3,"Les Eaux Minérales d'oulmès,Sidi Ali",6111035000058,Eau minérale naturelle,241.0,1190.0,15.0,4.10,6.1,1.40,5.7,2.5,65.00000,26.000000,1.0,0.0,0.00000,0.273333,Moderate,Low Sugar,No
4,AQUAFINA,6111252421568,اكوافينا,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00508,0.002032,4.0,0.0,0.00000,0.461538,Low,Low Sugar,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11793,Le Pain des Fleurs,3380380101830,Tartines craquantes noisette imp,391.0,1636.0,72.3,6.91,6.2,0.57,10.3,2.5,0.53000,0.212000,3.0,4.0,0.00000,0.095574,Moderate,Low Sugar,No
11794,Jordans,5010477342713,Jordans Country Crisp Honey And Nut,455.0,1190.0,61.0,22.00,18.0,2.00,9.4,5.5,0.03000,0.012000,4.0,9.0,0.00000,0.360656,High,Moderate Sugar,Yes
11795,Sainsbury's,316286,Fat free greek style yogurt,51.0,1190.0,5.2,5.20,0.5,0.10,6.4,0.5,0.13750,0.055000,1.0,-1.0,0.00000,1.000000,Low,Low Sugar,No
11796,Good life,5012869541803,Spicy veg bean burgers,210.0,876.0,20.0,2.00,9.7,0.60,5.8,8.9,0.75000,0.300000,3.0,-3.0,63.36575,0.100000,Moderate,Low Sugar,No


In [9]:
df.shape

(11798, 20)

In [10]:
df.isnull().sum()

brand                                                    0
product_code                                             0
product_name                                             0
energy-kcal_value                                        0
energy-kj_value                                          0
carbohydrates_value                                      0
sugars_value                                             0
fat_value                                                0
saturated-fat_value                                      0
proteins_value                                           0
fiber_value                                              0
salt_value                                               0
sodium_value                                             0
nova-group                                               0
nutrition-score-fr                                       0
fruits-vegetables-nuts-estimate-from-ingredients_100g    0
sugar_to_carb_ratio                                     

In [2]:
import mysql.connector
import pandas as pd 

In [3]:
#======= DATABASE COMNNECTION =======
mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",
 database = "CHOCOCRUNCH_ANALYSIS"
)
mycursor = mydb.cursor(buffered=True)

In [13]:
# Create tables

mycursor.execute("""
CREATE TABLE IF NOT EXISTS product_info (
    product_code VARCHAR(255) PRIMARY KEY,
    product_name TEXT,
    brand TEXT
)
""")
mycursor.execute("""
CREATE TABLE IF NOT EXISTS nutrient_info (
    product_code VARCHAR(255),
    `energy-kcal_value` FLOAT,
    `energy-kj_value` FLOAT,
    carbohydrates_value FLOAT,
    sugars_value FLOAT,
    fat_value FLOAT,
    `saturated-fat_value` FLOAT,
    proteins_value FLOAT,
    fiber_value FLOAT,
    salt_value FLOAT,
    sodium_value FLOAT,
    `fruits-vegetables-nuts-estimate-from-ingredients_100g` FLOAT,
    `nutrition-score-fr` INTEGER,
    `nova-group` INTEGER,
    FOREIGN KEY(product_code) REFERENCES product_info(product_code)
)
""")
mycursor.execute("""
CREATE TABLE IF NOT EXISTS derived_metrics (
    product_code VARCHAR(255),
    sugar_to_carb_ratio FLOAT,
    calorie_category TEXT,
    sugar_category TEXT,
    is_ultra_processed TEXT,
    FOREIGN KEY(product_code) REFERENCES product_info(product_code)
)
""")

In [14]:
# First insert all product_info
for _, row in df.iterrows():
    mycursor.execute("""
        INSERT INTO product_info (product_code, product_name, brand)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE product_name=VALUES(product_name), brand=VALUES(brand)
    """, (row['product_code'], row['product_name'], row['brand']))
mydb.commit()

# Then insert nutrient_info
for _, row in df.iterrows():
    mycursor.execute("""
        INSERT INTO nutrient_info (
            product_code, `energy-kcal_value`, `energy-kj_value`,
            carbohydrates_value, sugars_value, fat_value, `saturated-fat_value`,
            proteins_value, fiber_value, salt_value, sodium_value,
            `fruits-vegetables-nuts-estimate-from-ingredients_100g`,
            `nutrition-score-fr`, `nova-group`
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['product_code'], row['energy-kcal_value'], row['energy-kj_value'],
        row['carbohydrates_value'], row['sugars_value'], row['fat_value'], row['saturated-fat_value'],
        row['proteins_value'], row['fiber_value'], row['salt_value'], row['sodium_value'],
        row['fruits-vegetables-nuts-estimate-from-ingredients_100g'],
        row['nutrition-score-fr'], row['nova-group']
    ))
mydb.commit()

# Then insert derived_metrics
for _, row in df.iterrows():
    mycursor.execute("""
        INSERT INTO derived_metrics (
            product_code, sugar_to_carb_ratio, calorie_category, sugar_category, is_ultra_processed
        ) VALUES (%s, %s, %s, %s, %s)
    """, (
        row['product_code'], row['sugar_to_carb_ratio'],
        row['calorie_category'], row['sugar_category'], row['is_ultra_processed']
    ))
mydb.commit()

In [11]:

# Helper to display query results as DataFrame
def to_df(query):
    mycursor.execute(query)
    rows = mycursor.fetchall()
    cols = [desc[0] for desc in mycursor.description]
    return pd.DataFrame(rows, columns=cols)

# =========================================
# 📑 product_info queries
# =========================================
Pq1=to_df("""SELECT brand, COUNT(*) AS product_count FROM product_info GROUP BY brand;""")

Pq2=to_df("""SELECT brand, COUNT(DISTINCT product_code) AS unique_product_count FROM product_info GROUP BY brand;""")

Pq3=to_df("""SELECT brand, COUNT(*) AS product_count FROM product_info GROUP BY brand ORDER BY product_count DESC LIMIT 5;""")

Pq4=to_df("""SELECT * FROM product_info WHERE product_name IS NULL OR product_name = '';""")

Pq5=to_df("""SELECT COUNT(DISTINCT brand) AS unique_brands FROM product_info;""")

Pq6=to_df("""SELECT * FROM product_info WHERE product_code LIKE '3%';""")

# =========================================
# 📑 nutrient_info queries
# =========================================
Nq1=to_df("""SELECT product_code, `energy-kcal_value` FROM nutrient_info ORDER BY `energy-kcal_value` DESC LIMIT 10;""")

Nq2=to_df("""SELECT `nova-group`, AVG(sugars_value) AS avg_sugars FROM nutrient_info GROUP BY `nova-group`;""")

Nq3=to_df("""SELECT COUNT(*) AS high_fat_products FROM nutrient_info WHERE fat_value > 20;""")

Nq4=to_df("""SELECT product_code, AVG(carbohydrates_value) AS avg_carbs FROM nutrient_info GROUP BY product_code;""")

Nq5=to_df("""SELECT product_code, sodium_value FROM nutrient_info WHERE sodium_value > 1;""")

Nq6=to_df("""SELECT COUNT(*) AS non_zero_fvn FROM nutrient_info WHERE `fruits-vegetables-nuts-estimate-from-ingredients_100g` > 0;""")

Nq7=to_df("""SELECT product_code, `energy-kcal_value` FROM nutrient_info WHERE `energy-kcal_value` > 500;""")

# =========================================
# 📑 derived_metrics queries
# =========================================
Dq1=to_df("""SELECT calorie_category, COUNT(*) AS product_count FROM derived_metrics GROUP BY calorie_category;""")

Dq2=to_df("""SELECT COUNT(*) AS high_sugar_products FROM derived_metrics WHERE sugar_category = 'High Sugar';""")

Dq3=to_df("""SELECT AVG(sugar_to_carb_ratio) AS avg_ratio FROM derived_metrics WHERE calorie_category = 'High Calorie';""")

Dq4=to_df("""SELECT product_code FROM derived_metrics WHERE calorie_category = 'High Calorie' AND sugar_category = 'High Sugar';""")

Dq5=to_df("""SELECT COUNT(*) AS ultra_processed_products FROM derived_metrics WHERE is_ultra_processed = 'Yes';""")

Dq6=to_df("""SELECT product_code, sugar_to_carb_ratio FROM derived_metrics WHERE sugar_to_carb_ratio > 0.7;""")

Dq7=to_df("""SELECT calorie_category, AVG(sugar_to_carb_ratio) AS avg_ratio FROM derived_metrics GROUP BY calorie_category;""")

# =========================================
# 📑 Join Queries
# =========================================
Jq1 = to_df("""SELECT p.brand, COUNT(*) AS high_calorie_count
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.calorie_category = 'High Calorie'
GROUP BY p.brand
ORDER BY high_calorie_count DESC
LIMIT 5;""")

Jq2 = to_df("""SELECT d.calorie_category, AVG(n.`energy-kcal_value`) AS avg_energy
FROM nutrient_info n
JOIN derived_metrics d ON n.product_code = d.product_code
GROUP BY d.calorie_category;""")

Jq3 = to_df("""SELECT p.brand, COUNT(*) AS ultra_processed_count
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.is_ultra_processed = 'Yes'
GROUP BY p.brand;""")

Jq4 = to_df("""SELECT p.product_code, p.brand, d.calorie_category, d.sugar_category
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.calorie_category = 'High Calorie'
  AND d.sugar_category = 'High Sugar';""")

Jq5 = to_df("""SELECT p.brand, AVG(n.sugars_value) AS avg_sugars
FROM product_info p
JOIN nutrient_info n ON p.product_code = n.product_code
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.is_ultra_processed = 'Yes'
GROUP BY p.brand;""")

Jq6 = to_df("""SELECT d.calorie_category, COUNT(*) AS product_count
FROM nutrient_info n
JOIN derived_metrics d ON n.product_code = d.product_code
WHERE n.`fruits-vegetables-nuts-estimate-from-ingredients_100g` > 0
GROUP BY d.calorie_category;""")

Jq7 = to_df("""SELECT d.product_code, d.sugar_to_carb_ratio, d.calorie_category, d.sugar_category
FROM derived_metrics d
ORDER BY d.sugar_to_carb_ratio DESC
LIMIT 5;""")

In [15]:
display(Dq4)
display(Jq6)

,product_code


,calorie_category,product_count
0,High,1481
1,Low,2943
2,Moderate,1903


In [34]:
display(Pq1, Pq2, Pq3, Pq4, Pq5, Pq6)
display(Nq1, Nq2, Nq3, Nq4, Nq5, Nq6, Nq7)
display(Dq1, Dq2, Dq3, Dq4, Dq5, Dq6, Dq7)
display(Jq1, Jq2, Jq3, Jq4, Jq5, Jq6, Jq7)

,brand,product_count
0,"07 x Aldi 09.25, 09 x Aldi 09.25, 11 x Aldi 09...",1
1,"1540 Vestby, Norwegen, Oluf Lorentzen AS, Tove...",1
2,"166, 43122 Parma - Italy, Barilla, Barilla - B...",1
3,"1664 Blonde, Kronenbourg",1
4,2 KEEP NATURAL BITES,1
...,...,...
3517,"شرقي,Chergui",1
3518,كنور,1
3519,كوسومار,1
3520,لاسيگون,1


,brand,unique_product_count
0,"07 x Aldi 09.25, 09 x Aldi 09.25, 11 x Aldi 09...",1
1,"1540 Vestby, Norwegen, Oluf Lorentzen AS, Tove...",1
2,"166, 43122 Parma - Italy, Barilla, Barilla - B...",1
3,"1664 Blonde, Kronenbourg",1
4,2 KEEP NATURAL BITES,1
...,...,...
3517,"شرقي,Chergui",1
3518,كنور,1
3519,كوسومار,1
3520,لاسيگون,1


,brand,product_count
0,HACENDADO,384
1,Tesco,293
2,Carrefour,167
3,Bjorg,108
4,HEINZ,107


,product_code,product_name,brand


,unique_brands
0,3522


...

,brand,ultra_processed_count
0,"07 x Aldi 09.25, 09 x Aldi 09.25, 11 x Aldi 09...",1
1,2 KEEP NATURAL BITES,1
2,"25l, Red Bull, Zero 0, RedBull",1
3,2KEEP,2
4,"3D's Bugles, Bénénuts, Bénénuts 3D's Bugles, L...",1
...,...,...
2375,زيت الزيتون واد سوس,1
2376,سلطان,1
2377,كنور,1
2378,كوسومار,1


,product_code,brand,calorie_category,sugar_category


,brand,avg_sugars
0,"07 x Aldi 09.25, 09 x Aldi 09.25, 11 x Aldi 09...",5.00
1,2 KEEP NATURAL BITES,26.00
2,"25l, Red Bull, Zero 0, RedBull",0.00
3,2KEEP,19.05
4,"3D's Bugles, Bénénuts, Bénénuts 3D's Bugles, L...",4.00
...,...,...
2375,زيت الزيتون واد سوس,0.00
2376,سلطان,0.00
2377,كنور,0.50
2378,كوسومار,100.00


,calorie_category,product_count
0,High,1481
1,Low,2943
2,Moderate,1903


,product_code,sugar_to_carb_ratio,calorie_category,sugar_category
0,5000442007617,51250.0000,Low,Low Sugar
1,41500007229,87.2340,Low,Low Sugar
2,20002329,41.0000,Moderate,Low Sugar
3,6111184003283,29.2857,Low,Low Sugar
4,8402001021428,20.5000,Low,Low Sugar
